In [1]:
import os
import pandas as pd
import sklearn
from sklearn import preprocessing
import numpy as np

In [2]:
os.chdir('/Users/theophile/Desktop/ECP/Machine Learning/Sales Prediction/')

In [3]:
train=pd.read_csv('sales_train.csv')
items=pd.read_csv('items.csv')
item_cat=pd.read_csv('item_categories.csv')
shops=pd.read_csv('shops.csv')

In [4]:
#Caractéristiques du dataframe
def car(db):
    L=dict()
    for c in db.columns:
        df=db[c]
        L[c]=[len(df)-df.count(),df.min(),df.max()]
    return(L)

#### Mise en forme des données
Les infos utiles sont : date, boutique, article, catégorie, prix.<br>
L'objectif est de prédire les ventes mensuelles.

In [5]:
# train avec la catégorie du produit en plus
new=train.join(items,on="item_id",rsuffix="-") 
#on enlève item_id qui est redondant
new=new.drop(['item_id-'],axis=1) 

#On va regarder les ventes mensuelles par (produit, boutique, mois)
sauvegarde=new[['date_block_num','shop_id','item_id','item_category_id','item_price']]
new=new.drop(['item_category_id','item_price'],axis=1)
brand=new.groupby(['date_block_num','shop_id','item_id'],as_index=False).sum() 
brand.head()

,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0


In [6]:
sauvegarde = sauvegarde.drop_duplicates(['date_block_num','shop_id','item_id'])
start = pd.merge(brand, sauvegarde,  how='left', left_on=['date_block_num','shop_id','item_id'], right_on = ['date_block_num','shop_id','item_id'])
start.head()

,date_block_num,shop_id,item_id,item_cnt_day,item_category_id,item_price
0,0,0,32,6.0,40,221.0
1,0,0,33,3.0,37,347.0
2,0,0,35,1.0,40,247.0
3,0,0,43,1.0,40,221.0
4,0,0,51,2.0,57,130.0


# Premier modèle : régression linéaire (sans temporalité)
Régression linéaire globale : l (faut-il virer catégorie et shop d'ailleurs et l'omega de l'alpha dans la suite <em> ???)<br>
Régression linéaire par shop : $\delta_{i}$ <br>
Régression linéaire par item_category :  $\zeta_{j}$ <br>
Prédiction : L <br>


$ L = \alpha^{*}  l + \beta^{*}  \sum_{i \in Shops} \delta_{i} + \gamma^{*} \sum_{j \in Cats} \zeta_{j} $ <br>
Avec $ (\alpha^{*},\beta^{*}, \gamma^{*}) = argmin \sum_{x € X} (L_{(\alpha,\beta, \gamma)}(x) - y)^2 $

In [7]:
#On ne prend pas en compte le temps
ONE=start.drop(['date_block_num'],axis=1)
#target=rel['item_cnt_day']
#factors=rel.drop('item_cnt_day',axis=1)

In [8]:
def divide(db,col):#découpe le dataframe suivant les valeurs de la colonne en argument
    diff=db[col].unique()
    l_ddb=[]
    for x in diff:
        l_ddb.append(db[db[col]==x])
    return(l_ddb)

def shaping(db,L_col_cr): #centre-réduit les colonnes dans la LISTE L_col_cr
    for c in L_col_cr:
        db[c]=preprocessing.scale(db[c])

In [9]:
shaping(ONE,['item_price'])
ONE.columns=['shop','item','month_sales','category','price']
ONE.head()

,shop,item,month_sales,category,price
0,0,32,6.0,40,-0.367207
1,0,33,3.0,37,-0.286090
2,0,35,1.0,40,-0.350469
3,0,43,1.0,40,-0.367207
4,0,51,2.0,57,-0.425791


In [115]:
L_db_shops=divide(ONE,'shop')
L_db_cat=divide(ONE,'category')

##catégories trop peu fournies
for i in range(len(L_db_cat)):
    if len(L_db_cat[i]) <= 50:
        print(i,len(L_db_cat[i]))
print("------")
##boutiques avec trop peu de ventes
for i in range(len(L_db_shops)):
    if len(L_db_shops[i]) <= 50:
        print(i)

49 15
52 9
54 3
55 2
56 6
57 1
58 3
66 3
72 1
76 21
77 8
78 23
80 2
83 9
------


In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
# RL par boutique
shops_models=[0 for i in range (len(L_db_shops))]
for l in L_db_shops:
    X=l.drop(['month_sales'],axis=1) #shop est le même partout mais au moins le modèle est compatible avec les dimensions de base
    Y=l['month_sales']
    shops_models[int( l['shop'].iloc[0] )] = LinearRegression().fit(X, Y)

In [13]:
# RL par catégorie
cat_models=[0 for i in range (len(L_db_cat))]
for l in L_db_cat:
    X=l.drop(['month_sales'],axis=1) #cat est le même partout mais au moins le modèle est compatible avec les dimensions de base
    Y=l['month_sales']
    cat_models[int(l['category'].iloc[0])] = LinearRegression().fit(X, Y)

#### Idée plus simple pour calculer les prédictions du modèle pondéré
$ L_{(\alpha, \beta, \gamma)}(x) =\sum_{k} (\alpha  l_{k} + \beta \delta^{i}_{k} + \gamma \zeta^{j}_{k}) x_{k} $

In [14]:
X=ONE.drop(['month_sales'],axis=1) #shop est le même partout mais au moins le modèle est compatible avec les dimensions de base
Y=ONE['month_sales']
global_model = LinearRegression().fit(X, Y)


In [18]:
pred_by_shop=[]
pred_by_cat=[]
pred_glob = []
i=1
L=[global_model, shops_models, cat_models]
for n in X.index:
    x=np.array(X.iloc[n]).reshape(1,-1)
    pred_glob.append(L[0].predict(x))
    pred_by_shop.append(L[1][X['shop'].iloc[n]].predict(x))
    pred_by_cat.append(L[2][X['category'].iloc[n]].predict(x))
    if n>=i*len(X.index)/100:
        sys.stdout.flush()
        print("progression : {} %".format(i))
        i+=1

progression : 1 %
progression : 2 %
progression : 3 %
progression : 4 %
progression : 5 %
progression : 6 %
progression : 7 %
progression : 8 %
progression : 9 %
progression : 10 %
progression : 11 %
progression : 12 %
progression : 13 %
progression : 14 %
progression : 15 %
progression : 16 %
progression : 17 %
progression : 18 %
progression : 19 %
progression : 20 %
progression : 21 %
progression : 22 %
progression : 23 %
progression : 24 %
progression : 25 %
progression : 26 %
progression : 27 %
progression : 28 %
progression : 29 %
progression : 30 %
progression : 31 %
progression : 32 %
progression : 33 %
progression : 34 %
progression : 35 %
progression : 36 %
progression : 37 %
progression : 38 %
progression : 39 %
progression : 40 %
progression : 41 %
progression : 42 %
progression : 43 %
progression : 44 %
progression : 45 %
progression : 46 %
progression : 47 %
progression : 48 %
progression : 49 %
progression : 50 %
progression : 51 %
progression : 52 %
progression : 53 %
pr

In [99]:
pred_by_shop=list(map(float,pred_by_shop))
pred_by_cat=list(map(float,pred_by_cat))
pred_glob=list(map(float,pred_glob))


DIFF=pd.DataFrame({'global prediction' : pred_glob,
                   'prediction by shop': pred_by_shop,
              'prediction by category' : pred_by_cat,
              'target' : Y})

DIFF.head()

,global prediction,prediction by shop,prediction by category,target
0,2.270487,2.846691,1.472259,6.0
1,2.300942,2.941759,1.510901,3.0
2,2.273559,2.848876,1.616814,1.0
3,2.270401,2.846312,1.472377,1.0
4,2.171883,2.361956,1.142932,2.0


In [116]:
Models=DIFF.drop('target',axis=1)
target=DIFF['target']
reg=LinearRegression().fit(Models,target)
reg.coef_

array([-0.76962777,  0.77387548,  0.99575229])

# Il faut regarder si avec ces coefs la régression est meilleure que la prédiction par catégorie (on dirait que les 2 premiers s'annulent)

In [117]:
#Si on veut que la somme des coefs fasse 1
import copy
def calc(poids,db):
    a,b=poids[0],poids[1]
    U=copy.deepcopy(db)
    U['global prediction']=U['global prediction'].apply(lambda x : a*x)
    U['prediction by shop']=U['prediction by shop'].apply(lambda x : b*x)
    U['prediction by category']=U['prediction by category'].apply(lambda x : (1-a-b)*x )
    U['resultat']=(U['global prediction']+U['prediction by shop']+U['prediction by category']-U['target'])**2
    return(U['resultat'].sum())

In [103]:
[x,y,z]=[0,0,1]
Min=calc([x,y],DIFF)
for i in range (1,11):
    print("progression : {}%".format(i*10))
    for j in range (1, 11):
        if i+j<=10:
            k=10-i-j
            image=calc([i/10,j/10],DIFF)
            print(image)
            if image<=Min:
                Min=image
                [x,y,z]=[i/10,j/10,k/10]
print(Min,[x,y,z])

progression : 10%
89588776.22363989
91072517.60700826
93202156.68944976
95977693.47096434
99399127.95155223
103466460.13121325
108179690.00994739
113538817.58775494
119543842.86463547
progression : 20%
91178995.72706637
93296725.45820642
96060352.88841984
99469878.01770635
103525300.84606592
108226621.3734989
113573839.60000487
119566955.5255841
progression : 30%
93406702.66759573
96158420.74650772
99556036.52449279
103599550.00155115
108288961.17768262
113624270.05288745
119605476.62716521
progression : 40%
96271897.04522802
99657603.47191179
103689207.59766878
108366709.42249893
113690108.94640233
119659406.16937873
progression : 50%
99774578.85996316
103794273.63441882
108459866.10794756
113771356.28054965
119728744.15222475
progression : 60%
103914748.11180125
108568431.23402874
113868012.0553294
119813490.57570326
progression : 70%
108692404.80074237
113980076.27074163
119913645.43981408
progression : 80%
114107548.9267863
120029208.7445574
progression : 90%
120160180.4899331
prog

## SUPER : la meilleure solution est alpha = 0, beta=0, gamma =1 (i.e. on ne se sert que de la prédiction par catégorie)

# Deuxième modèle : réseau de neurones